## Libraries

In [1]:
import numpy as np
import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

## Model

In [2]:
class JudgeModel(nn.Module):
    def __init__(
        self,
        input_dim: int,
        hidden_dim: int = 64,
        num_classes: int = 3
    ):
        super().__init__()

        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, num_classes)
        )

    def forward(self, x):
        logits = self.net(x)
        probs = F.softmax(logits, dim=-1)

        return {
            "logits": logits,
            "probs": probs
        }

## Dataset

In [3]:
class JudgeDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X)
        self.y = torch.tensor(y)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

## Generation synthetic data

In [4]:
VERDICTS = {"publish": 0, "revise": 1, "reject": 2}
RISK_LABELS = ["scam", "adult", "illegal", "spam", "low_info"]


def build_judge_features(auditor, quality, meta=None):
    features = [
        auditor["risk_score"],
        auditor.get("uncertainty", 0.0),
        quality["quality_score"],
        quality.get("confidence", 1.0),
    ]

    # Quality aspects
    for k in ["informativeness", "clarity", "completeness", "persuasion"]:
        features.append(quality["aspects"].get(k, 0.0))

    # Optional meta-features
    if meta:
        features.append(meta.get("text_length", 0) / 1000)
        features.append(meta.get("embedding_norm", 0.0))

    return np.array(features, dtype="float32")


def generate_fake_auditor():
    risk = np.random.beta(2, 5)
    label_probs = np.random.dirichlet(np.ones(len(RISK_LABELS)))
    entropy = -(label_probs * np.log(label_probs + 1e-8)).sum()

    return {
        "risk_score": float(risk),
        "risk_labels": dict(zip(RISK_LABELS, label_probs)),
        "uncertainty": float(entropy)
    }


def generate_fake_quality():
    aspects = {
        "informativeness": np.random.uniform(0.2, 1.0),
        "clarity": np.random.uniform(0.2, 1.0),
        "completeness": np.random.uniform(0.2, 1.0),
        "persuasion": np.random.uniform(0.2, 1.0),
    }

    quality = np.mean(list(aspects.values()))
    confidence = 1 - np.std(list(aspects.values()))

    return {
        "quality_score": float(quality),
        "aspects": aspects,
        "confidence": float(confidence)
    }


def generate_judge_sample():
    auditor = generate_fake_auditor()
    quality = generate_fake_quality()

    meta = {
        "text_length": np.random.randint(20, 2000),
        "embedding_norm": np.random.uniform(5, 15)
    }

    features = build_judge_features(auditor, quality, meta)

    if auditor["risk_score"] > 0.7:
        label = VERDICTS["reject"]
    elif quality["quality_score"] > 0.75 and auditor["risk_score"] < 0.3:
        label = VERDICTS["publish"]
    else:
        label = VERDICTS["revise"]

    return np.array(features, dtype="float32"), label


def generate_judge_dataset(n=5000):
    X, y = [], []
    for _ in range(n):
        f, l = generate_judge_sample()
        X.append(f)
        y.append(l)
    return X, y

## Train function

In [5]:
def train_judge(dataset, input_dim: int):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    model = JudgeModel(input_dim=input_dim).to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)
    criterion = torch.nn.CrossEntropyLoss()
    epochs = 10

    loader = DataLoader(dataset, batch_size=32, shuffle=True)

    model.train()
    for epoch in range(epochs):
        epoch_loss = 0
        loop = tqdm.tqdm(loader, desc=f"Epoch {epoch + 1}")

        for x, y in loop:
            x, y = x.to(device), y.to(device)
            
            out = model(x)
            loss = criterion(out["logits"], y)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()
            loop.set_postfix(loss=loss.item())

        print(f"Epoch {epoch + 1}: loss={epoch_loss / len(loader):.4f}")

    return model

## Train

In [6]:
X, y = generate_judge_dataset()
dataset = JudgeDataset(X, y)

model = train_judge(dataset, input_dim=X[0].shape[0])
torch.save(model.state_dict(), "judge.pt")

/var/folders/z9/19wy68_x711cm327qjf79_pc0000gn/T/ipykernel_17170/3878342614.py:3: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:256.)
  self.X = torch.tensor(X)
Epoch 1: 100%|██████████| 157/157 [00:00<00:00, 502.09it/s, loss=0.099]


Epoch 1: loss=0.3266


Epoch 2: 100%|██████████| 157/157 [00:00<00:00, 884.62it/s, loss=0.0754]


Epoch 2: loss=0.2685


Epoch 3: 100%|██████████| 157/157 [00:00<00:00, 864.62it/s, loss=0.0639]


...

Epoch 8: loss=0.1239


Epoch 9: 100%|██████████| 157/157 [00:00<00:00, 909.05it/s, loss=0.28] 


Epoch 9: loss=0.1222


Epoch 10: 100%|██████████| 157/157 [00:00<00:00, 911.67it/s, loss=0.0651]


Epoch 10: loss=0.1113
